<a href="https://colab.research.google.com/github/rll/deepul/blob/master/homeworks/solutions/hw2_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Getting Started

## Overview
This semester, all homeworks will be conducted through Google Colab notebooks. All code for the homework assignment will be written and run in this notebook. Running in Colab will automatically provide a GPU, but you may also run this notebook locally by following [these instructions](https://research.google.com/colaboratory/local-runtimes.html) if you wish to use your own GPU.

You will save images in the notebooks to use and fill out a given LaTeX template which will be submitted to Gradescope, along with your notebook code.

## Using Colab
On the left-hand side, you can click the different icons to see a Table of Contents of the assignment, as well as local files accessible through the notebook.

Make sure to go to **Runtime -> Change runtime type** and select **GPU** as the hardware accelerator. This allows you to use a GPU. Run the cells below to get started on the assignment. Note that a session is open for a maximum of 12 hours, and using too much GPU compute may result in restricted access for a short period of time. Please start the homework early so you have ample time to work.

**If you loaded this notebook from clicking "Open in Colab" from github, you will need to save it to your own Google Drive to keep your work.**

## General Tips
In each homework problem, you will implement an flow model and run it on one to two datasets. We provide the expected outputs to help as a sanity check.

Feel free to print whatever output (e.g. debugging code, training code, etc) you want, as the graded submission will be the submitted pdf with images.

After you complete the assignment, download all of the image outputted in the results/ folder and upload them to the figure folder in the given latex template.

Run the cells below to download and load up the starter code.

In [ ]:
# !if [ -d deepul ]; then rm -Rf deepul; fi
# !git clone https://github.com/rll/deepul.git
# !pip install ./deepul

In [ ]:
import warnings

from deepul.hw2_helper import *

warnings.filterwarnings("ignore")

# Question 1: 2D Data

In this question, we will train two variants of flow models on continuous 2D data. 

Execute the cell below to visualize our datasets

In [ ]:
visualize_q1_data(dset_type=1)
visualize_q1_data(dset_type=2)

## Part (a) Autoregressive Flow [15pt]

In an autoregressive flow, we learn the transformation $z_i = f(x_i ; x_{1:i-1})$. The log-likelihood is then $\log p_\theta(x) = \sum_{i=1}^d p(z_i) + log |\frac{dz_i}{dx_i}|$ because the Jacobian is triangular. For each dimension, use the CDF of a mixture of Gaussians or Logistics to map to the latent space, which should be $z_i \sim \text{Unif}[0, 1]$ i.i.d..

Fit $p_\theta$ with maximum likelihood via stochastic gradient descent on the training set. Since this is a 2D problem, you can either learn $z_0 = f(x_0)$ and $z_1 = f(x_1; x_0)$ together with a MADE model, or with separate networks. Use your favorite version of stochastic gradient descent, and optimize your hyperparameters on a validation set of your choice.

**You will provide these deliverables**


1.   Over the course of training, record the average negative log-likelihood (nats / dim) of the training data (per minibatch) and test data (for your entire test set). Code is provided that automatically plots the training curves. 
2.   Report the final test set performance of your final model
3. Show a heatmap of the densities learned by your model
4. Show what the training set looks like when mapped to the latent space. 




### Solution
Fill out the function below and return the necessary arguments. Feel free to create more cells if need be.

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from tqdm import tqdm_notebook, trange

import deepul.pytorch_util as ptu


def train(model, train_loader, optimizer):
    model.train()
    losses = []
    for x in train_loader:
        x = x.to(ptu.device).float().contiguous()
        loss = model.nll(x)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return losses


def eval_loss(model, data_loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for x in data_loader:
            x = x.to(ptu.device).float().contiguous()
            loss = model.nll(x)
            total_loss += loss * x.shape[0]
        avg_loss = total_loss / len(data_loader.dataset)
    return avg_loss.item()


def train_epochs(model, train_loader, test_loader, train_args):
    epochs, lr = train_args["epochs"], train_args["lr"]
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_losses, test_losses = [], []
    test_loss = eval_loss(model, test_loader)
    test_losses.append(test_loss)  # loss at init
    for epoch in tqdm_notebook(range(epochs), desc="Epoch", leave=False):
        model.train()
        epoch_train_losses = train(model, train_loader, optimizer)
        train_losses.extend(epoch_train_losses)

        test_loss = eval_loss(model, test_loader)
        test_losses.append(test_loss)

    return train_losses, test_losses

In [ ]:
import copy

import matplotlib.pyplot as plt
from scipy.stats import norm
from torch.distributions.normal import Normal
from torch.distributions.uniform import Uniform

ptu.set_gpu_mode(False)


class MixtureCDFFlow(nn.Module):
    def __init__(
        self,
        base_dist="uniform",
        mixture_dist="gaussian",
        n_components=4,
        plot_bounds=(-3, 3),
    ):
        super().__init__()
        self.composition = False
        if base_dist == "uniform":
            self.base_dist = Uniform(0.0, 1.0)
        else:
            raise NotImplementedError

        self.loc = nn.Parameter(torch.randn(n_components), requires_grad=True)
        self.log_scale = nn.Parameter(torch.zeros(n_components), requires_grad=True)
        self.weight_logits = nn.Parameter(torch.zeros(n_components), requires_grad=True)
        if mixture_dist == "gaussian":
            self.mixture_dist = Normal
        elif mixture_dist == "logistic":
            raise NotImplementedError
        self.n_components = n_components
        self.plot_bounds = plot_bounds

    def flow(self, x):
        # z = cdf of x
        weights = (
            F.softmax(self.weight_logits, dim=0).unsqueeze(0).repeat(x.shape[0], 1)
        )
        z = (
            self.mixture_dist(self.loc, self.log_scale.exp()).cdf(
                x.unsqueeze(1).repeat(1, self.n_components)
            )
            * weights
        ).sum(dim=1)
        z = torch.clamp(z, 0, 1 - 1e-5)

        # log_det = log dz/dx = log pdf(x)
        log_det = (
            (
                self.mixture_dist(self.loc, self.log_scale.exp())
                .log_prob(x.unsqueeze(1).repeat(1, self.n_components))
                .exp()
                * weights
            )
            .sum(dim=1)
            .log()
        )

        return z, log_det

    def log_prob(self, x):
        z, log_det = self.flow(x)
        return self.base_dist.log_prob(z) + log_det

    # Compute loss as negative log-likelihood
    def nll(self, x):
        return -self.log_prob(x).mean()


class MLP(nn.Module):
    def __init__(self, input_size, n_hidden, hidden_size, output_size):
        super().__init__()
        layers = []
        for _ in range(n_hidden):
            layers.append(nn.Linear(input_size, hidden_size))
            layers.append(nn.ReLU())
            input_size = hidden_size
        layers.append(nn.Linear(hidden_size, output_size))
        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        return self.layers(x)


class AutoregressiveFlow(nn.Module):
    """
    Assumes that we're modeling a 2D distribution
    """

    def __init__(
        self,
        dim1_mixture_dist="gaussian",
        dim1_n_components=5,
        dim2_mixture_dist="gaussian",
        dim2_n_components=5,
        mlp_hidden_size=64,
        mlp_n_layers=3,
    ):
        super().__init__()
        self.dim1_flow = MixtureCDFFlow(
            mixture_dist=dim1_mixture_dist, n_components=dim1_n_components
        ).to(ptu.device)
        self.mlp = MLP(1, mlp_n_layers, mlp_hidden_size, dim2_n_components * 3).to(
            ptu.device
        )
        self.dim2_n_components = dim2_n_components
        self.base_dist = Uniform(ptu.tensor(0.0), ptu.tensor(1.0))
        if dim2_mixture_dist == "gaussian":
            self.mixture_dist = Normal
        elif dim2_mixture_dist == "logistic":
            raise NotImplementedError

    def flow(self, x):
        # individually flow on each dim
        x1, x2 = torch.chunk(x, 2, dim=1)
        # dim 1
        z1, log_det1 = self.dim1_flow.flow(x1.squeeze())

        # dim1
        loc, log_scale, weight_logits = torch.chunk(self.mlp(x1), 3, dim=1)
        weights = F.softmax(weight_logits, dim=1)
        z2 = (
            self.mixture_dist(loc, log_scale.exp()).cdf(
                x2.repeat(1, self.dim2_n_components)
            )
            * weights
        ).sum(dim=1)

        z2 = torch.clamp(z2, 0, 1 - 1e-5)
        log_det2 = (
            (
                self.mixture_dist(loc, log_scale.exp())
                .log_prob(x2.repeat(1, self.dim2_n_components))
                .exp()
                * weights
            )
            .sum(dim=1)
            .log()
        )
        return torch.cat([z1.unsqueeze(1), z2.unsqueeze(1)], dim=1), torch.cat(
            [log_det1.unsqueeze(1), log_det2.unsqueeze(1)], dim=1
        )

    def log_prob(self, x):
        z, log_det = self.flow(x)
        return (self.base_dist.log_prob(z) + log_det).mean(
            dim=1
        )  # shape: [batch_size, dim]

    def nll(self, x):
        return -self.log_prob(x).mean()

In [ ]:
def q1_a(train_data, test_data, dset_id):
    """
    train_data: An (n_train, 2) numpy array of floats in R^2
    test_data: An (n_test, 2) numpy array of floats in R^2
    dset_id: An identifying number of which dataset is given (1 or 2). Most likely
               used to set different hyperparameters for different datasets, or
               for plotting a different region of densities

    Returns
    - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
    - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
    - a numpy array of size (?,) of probabilities with values in [0, +infinity).
        Refer to the commented hint.
    - a numpy array of size (n_train, 2) of floats in [0,1]^2. This represents
        mapping the train set data points through our flow to the latent space.
    """

    """ YOUR CODE HERE """
    # create data loaders
    loader_args = dict(batch_size=128, shuffle=True)
    train_loader = data.DataLoader(train_data, **loader_args)
    test_loader = data.DataLoader(test_data, **loader_args)

    # model
    ar_flow = AutoregressiveFlow()

    # train
    train_losses, test_losses = train_epochs(
        ar_flow, train_loader, test_loader, dict(epochs=100, lr=5e-3)
    )

    # heatmap
    dx, dy = 0.025, 0.025
    if dset_id == 1:  # face
        x_lim = (-4, 4)
        y_lim = (-4, 4)
    elif dset_id == 2:  # two moons
        x_lim = (-1.5, 2.5)
        y_lim = (-1, 1.5)
    y, x = np.mgrid[
        slice(y_lim[0], y_lim[1] + dy, dy), slice(x_lim[0], x_lim[1] + dx, dx)
    ]
    mesh_xs = ptu.FloatTensor(np.stack([x, y], axis=2).reshape(-1, 2))
    densities = np.exp(ptu.get_numpy(ar_flow.log_prob(mesh_xs)))

    # latents
    z, _ = ar_flow.flow(ptu.FloatTensor(train_data))
    latents = ptu.get_numpy(z)

    return train_losses, test_losses, densities, latents

### Results

Once you've implemented `q1_a`, execute the cells below to visualize and save your results



In [ ]:
q1_save_results(1, "a", q1_a)

In [ ]:
q1_save_results(2, "a", q1_a)

## Part (b) RealNVP [15pt]
In this part, we want to train a flow with the following structure: 
$(z_1, z_2) = (f_{\theta, 1} \circ \cdots \circ f_{\theta, n})$, 
where each $f_{\theta, i}$ is an affine transformation of 1 dimension, conditioned on the other. According to [Density Estimation Using Real NVP](https://arxiv.org/abs/1605.08803) Section 4.1, there's a particularly good way to parameterize the affine transformation:

Assuming that we're conditioning on $x_1$ and transforming $x_2$, we have 

- $z_1 = x_1$

- $ \text{log_scale} = \text{scale} \times tanh(g_{\theta, \text{scale}}(x_1)) + \text{scale_shift}$

- $z_2 = exp(\text{log_scale}) \times x_2 + g_{\theta, \text{shift}}(x_1)$

where $g_\theta$, $\text{scale}$, and $\text{scale_shift}$ are all learned parameters. The latent space should be $z \sim \mathcal N (0, I_2)$.

**Train a RealNVP model and provide the same set of corresponding deliverables as part (a)**

### Solution
Fill out the function below and return the necessary arguments. Feel free to create more cells if need be.

In [ ]:
class AffineTransform(nn.Module):
    def __init__(self, type, n_hidden=2, hidden_size=256):
        super().__init__()
        self.mask = self.build_mask(type=type)
        self.scale = nn.Parameter(torch.zeros(1), requires_grad=True)
        self.scale_shift = nn.Parameter(torch.zeros(1), requires_grad=True)
        self.mlp = MLP(
            input_size=2, n_hidden=n_hidden, hidden_size=hidden_size, output_size=2
        )

    def build_mask(self, type):
        # if type == "left", left half is a one
        # if type == right", right half is a one
        assert type in {"left", "right"}
        if type == "left":
            mask = ptu.FloatTensor([1.0, 0.0])
        elif type == "right":
            mask = ptu.FloatTensor([0.0, 1.0])
        else:
            raise NotImplementedError
        return mask

    def forward(self, x, reverse=False):
        # returns transform(x), log_det
        batch_size = x.shape[0]
        mask = self.mask.repeat(batch_size, 1)
        x_ = x * mask

        log_s, t = self.mlp(x_).split(1, dim=1)  # todo: fix
        log_s = self.scale * torch.tanh(log_s) + self.scale_shift
        t = t * (1.0 - mask)
        log_s = log_s * (1.0 - mask)

        if reverse:  # inverting the transformation
            x = (x - t) * torch.exp(-log_s)
        else:
            x = x * torch.exp(log_s) + t
        return x, log_s


class SigmoidFlow(nn.Module):
    def forward(self, x, reverse=False):
        if reverse:
            z = -torch.log(torch.reciprocal(x) - 1)
        else:
            z = torch.sigmoid(x)
        return z, z.log() + (1 - z).log()


class RealNVP(nn.Module):
    def __init__(self, transforms):
        super().__init__()

        self.prior = torch.distributions.Normal(
            torch.tensor(0.0).to(ptu.device), torch.tensor(1.0).to(ptu.device)
        )
        self.transforms = nn.ModuleList(transforms)

    def flow(self, x):
        # maps x -> z, and returns the log determinant (not reduced)
        z, log_det = x, torch.zeros_like(x)
        for op in self.transforms:
            z, delta_log_det = op.forward(z)
            log_det += delta_log_det
        return z, log_det

    def invert_flow(self, z):
        # z -> x (inverse of f)
        for op in reversed(self.transforms):
            z, _ = op.forward(z, reverse=True)
        return z

    def log_prob(self, x):
        z, log_det = self.flow(x)
        return torch.sum(log_det, dim=1) + torch.sum(self.prior.log_prob(z), dim=1)

    def sample(self, num_samples):
        z = self.prior.sample([num_samples, 2])
        return self.invert_flow(z)

    def nll(self, x):
        return -self.log_prob(x).mean()

In [ ]:
def q1_b(train_data, test_data, dset_id):
    """
    train_data: An (n_train, 2) numpy array of floats in R^2
    test_data: An (n_test, 2) numpy array of floats in R^2
    dset_id: An identifying number of which dataset is given (1 or 2). Most likely
               used to set different hyperparameters for different datasets, or
               for plotting a different region of densities

    Returns
    - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
    - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
    - a numpy array of size (?,) of probabilities with values in [0, +infinity).
        Refer to the commented hint.
    - a numpy array of size (n_train, 2) of floats in R^2. This represents
        mapping the train set data points through our flow to the latent space.
    """

    """ YOUR CODE HERE """
    # create data loaders
    loader_args = dict(batch_size=128, shuffle=True)
    train_loader = data.DataLoader(train_data, **loader_args)
    test_loader = data.DataLoader(test_data, **loader_args)

    # model
    real_nvp = RealNVP(
        [
            AffineTransform("left", n_hidden=2, hidden_size=64),
            AffineTransform("right", n_hidden=2, hidden_size=64),
            AffineTransform("left", n_hidden=2, hidden_size=64),
            AffineTransform("right", n_hidden=2, hidden_size=64),
            AffineTransform("left", n_hidden=2, hidden_size=64),
            AffineTransform("right", n_hidden=2, hidden_size=64),
        ]
    )

    # train
    train_losses, test_losses = train_epochs(
        real_nvp, train_loader, test_loader, dict(epochs=250, lr=5e-3)
    )

    # heatmap
    dx, dy = 0.025, 0.025
    if dset_id == 1:  # face
        x_lim = (-4, 4)
        y_lim = (-4, 4)
    elif dset_id == 2:  # two moons
        x_lim = (-1.5, 2.5)
        y_lim = (-1, 1.5)
    y, x = np.mgrid[
        slice(y_lim[0], y_lim[1] + dy, dy), slice(x_lim[0], x_lim[1] + dx, dx)
    ]
    mesh_xs = ptu.FloatTensor(np.stack([x, y], axis=2).reshape(-1, 2))
    densities = np.exp(ptu.get_numpy(real_nvp.log_prob(mesh_xs)))

    # latents
    z, _ = real_nvp.flow(ptu.FloatTensor(train_data))
    latents = ptu.get_numpy(z)

    return train_losses, test_losses, densities, latents

### Results

Once you've implemented `q1_b`, execute the cells below to visualize and save your results



In [ ]:
q1_save_results(1, "b", q1_b)

In [ ]:
q1_save_results(2, "b", q1_b)

# Question 2: Autoregressive Flows for Images [20pt]
We can scale up the autoregressive flows from 1(a) to high-dimensional image data. In this question, we will focus on the black-and-white shapes dataset from Homework 1.

In [ ]:
visualize_q2_data()

Adapt your PixelCNN implementation from Homework 1 and use it as an autoregressive flow model on the black-and-white shapes dataset. Using the base architecture from Homework 1 3(a), output a mixture of logistics or gaussians (instead of logits), whose CDF we will use as our flow for each dimension of x. 

Remember: you must dequantize the data and scale it between 0 and 1 for the autoregressive flow to have stable training. 

**You will provide these deliverables**

1.   Over the course of training, record the average negative log-likelihood (nats / dim) of the training data (per minibatch) and test data (for your entire test set). Code is provided that automatically plots the training curves. **Make sure to account for any rescaling you do in your nats/dim.**
2.   Report the final test set performance of your final model
3. 100 samples from your autoregressive flow. Our plotting code will show your samples, with and without clipping and flooring (to remove the quantization). 


### Solution
Fill out the function below and return the necessary arguments. Feel free to create more cells if need be.

In [ ]:
def train_with_dequantization_q2(model, train_loader, optimizer):
    model.train()
    losses = []
    for x in train_loader:
        x = x.to(ptu.device).float().contiguous()
        x += torch.distributions.Uniform(0.0, 0.5).sample(x.shape).to(ptu.device)
        loss = model.nll(x)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return losses


def train_epochs_q2(model, train_loader, test_loader, train_args):
    epochs, lr = train_args["epochs"], train_args["lr"]
    optimizer = optim.Adam(model.parameters(), lr=lr)

    train_losses, test_losses = [], []
    test_loss = eval_loss(model, test_loader)
    test_losses.append(test_loss)  # loss at init
    for epoch in tqdm_notebook(range(epochs), desc="Epoch", leave=False):
        model.train()
        epoch_train_losses = train_with_dequantization_q2(
            model, train_loader, optimizer
        )
        train_losses.extend(epoch_train_losses)

        test_loss = eval_loss(model, test_loader)
        test_losses.append(test_loss)

    return train_losses, test_losses

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.normal import Normal

ptu.set_gpu_mode(True)


class MaskConv2d(nn.Conv2d):
    def __init__(self, mask_type, *args, **kwargs):
        assert mask_type == "A" or mask_type == "B"
        super().__init__(*args, **kwargs)
        self.register_buffer("mask", torch.zeros_like(self.weight))
        self.create_mask(mask_type)

    def forward(self, input, cond=None):
        batch_size = input.shape[0]
        out = F.conv2d(
            input,
            self.weight * self.mask,
            self.bias,
            self.stride,
            self.padding,
            self.dilation,
            self.groups,
        )
        return out

    def create_mask(self, mask_type):
        k = self.kernel_size[0]
        self.mask[:, :, : k // 2] = 1
        self.mask[:, :, k // 2, : k // 2] = 1
        if mask_type == "B":
            self.mask[:, :, k // 2, k // 2] = 1


class ResBlock(nn.Module):
    def __init__(self, in_channels, **kwargs):
        super().__init__()
        self.block = nn.ModuleList(
            [
                nn.ReLU(),
                MaskConv2d("B", in_channels, in_channels // 2, 1, **kwargs),
                nn.ReLU(),
                MaskConv2d(
                    "B", in_channels // 2, in_channels // 2, 7, padding=3, **kwargs
                ),
                nn.ReLU(),
                MaskConv2d("B", in_channels // 2, in_channels, 1, **kwargs),
            ]
        )

    def forward(self, x, cond=None):
        out = x
        for layer in self.block:
            if isinstance(layer, MaskConv2d):
                out = layer(out, cond=cond)
            else:
                out = layer(out)
        return out + x


class LayerNorm(nn.LayerNorm):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def forward(self, x):
        x = x.permute(0, 2, 3, 1).contiguous()
        x_shape = x.shape
        x = super().forward(x)
        return x.permute(0, 3, 1, 2).contiguous()


class AutoregressiveFlowPixelCNN(nn.Module):
    def __init__(
        self,
        input_shape,
        n_filters=64,
        kernel_size=7,
        n_layers=5,
        use_resblock=False,
        n_components=2,
        **kwargs
    ):
        super().__init__()
        assert n_layers >= 2
        n_channels = input_shape[0]

        if use_resblock:
            block_init = lambda: ResBlock(n_filters, **kwargs)
        else:
            block_init = lambda: MaskConv2d(
                "B",
                n_filters,
                n_filters,
                kernel_size=kernel_size,
                padding=kernel_size // 2,
                **kwargs
            )

        model = nn.ModuleList(
            [
                MaskConv2d(
                    "A",
                    n_channels,
                    n_filters,
                    kernel_size=kernel_size,
                    padding=kernel_size // 2,
                    **kwargs
                )
            ]
        )
        for _ in range(n_layers):
            model.append(LayerNorm(n_filters))
            model.extend([nn.ReLU(), block_init()])
        model.extend([nn.ReLU(), MaskConv2d("B", n_filters, n_filters, 1, **kwargs)])
        model.extend(
            [
                nn.ReLU(),
                MaskConv2d("B", n_filters, n_components * 3 * n_channels, 1, **kwargs),
            ]
        )

        self.net = model
        self.input_shape = input_shape
        self.n_channels = n_channels
        self.n_components = n_components

    def forward(self, x, cond=None):
        batch_size = x.shape[0]
        out = x.float()
        for layer in self.net:
            if isinstance(layer, MaskConv2d) or isinstance(layer, ResBlock):
                out = layer(out, cond=cond)
            else:
                out = layer(out)
        return out.view(
            batch_size, 3 * self.n_components, *self.input_shape
        )  # todo: fix this line

    def nll(self, x, cond=None):
        loc, log_scale, weight_logits = torch.chunk(self.forward(x), 3, dim=1)
        weights = F.softmax(
            weight_logits, dim=1
        )  # .repeat(1, 1, self.n_components, 1, 1)
        log_det_jacobian = Normal(loc, log_scale.exp()).log_prob(
            x.unsqueeze(1).repeat(1, 1, self.n_components, 1, 1)
        )
        return -log_det_jacobian.mean()

    def sample(self, n, cond=None):
        samples = torch.zeros(n, *self.input_shape).cuda()
        with torch.no_grad():
            for r in range(self.input_shape[1]):
                for c in range(self.input_shape[2]):
                    for k in range(self.n_channels):
                        loc, log_scale, weight_logits = torch.chunk(
                            self.forward(samples), 3, dim=1
                        )
                        loc, log_scale, weight_logits = (
                            loc[:, :, k, r, c],
                            log_scale[:, :, k, r, c],
                            weight_logits[:, :, k, r, c],
                        )
                        probs = F.softmax(weight_logits, dim=1)
                        chosen_centers = torch.multinomial(probs, 1).squeeze(-1)
                        samples[:, k, r, c] = torch.normal(
                            loc[torch.arange(n), chosen_centers],
                            log_scale[torch.arange(n), chosen_centers].exp(),
                        )
        return samples.permute(0, 2, 3, 1).cpu().numpy()

In [ ]:
def q2(train_data, test_data):
    """
    train_data: A (n_train, H, W, 1) uint8 numpy array of binary images with values in {0, 1}
    test_data: A (n_test, H, W, 1) uint8 numpy array of binary images with values in {0, 1}
    H = W = 20
    Note that you should dequantize your train and test data, your dequantized pixels should all lie in [0,1]

    Returns
    - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
    - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
    - a numpy array of size (100, H, W, 1) of samples with values in [0, 1], where [0,0.5] represents a black pixel
        and [0.5,1] represents a white pixel. We will show your samples with and without noise.
    """

    """ YOUR CODE HERE """
    train_data = np.transpose(train_data, (0, 3, 1, 2)).astype(np.float32) / 2.0
    test_data = np.transpose(test_data, (0, 3, 1, 2)).astype(np.float32) / 2.0
    # train_data = train_data + np.random.uniform(high=0.5, size=train_data.shape)
    test_data = test_data + np.random.uniform(high=0.5, size=test_data.shape)

    H, W = 20, 20
    model = AutoregressiveFlowPixelCNN((1, H, W), n_layers=5, n_components=10).cuda()

    train_loader = data.DataLoader(train_data, batch_size=128, shuffle=True)
    test_loader = data.DataLoader(test_data, batch_size=128)
    train_losses, test_losses = train_epochs_q2(
        model, train_loader, test_loader, dict(epochs=20, lr=1e-3)
    )
    samples = model.sample(100)
    return train_losses + np.log(2), test_losses + np.log(2), samples

### Results

Once you've implemented `q2`, execute the cells below to visualize and save your results



In [ ]:
q2_save_results(q2)

# Question 3: RealNVP on Higher Dimensions


In this problem, you will train a flow model to invertibly map a high dimensional dataset of celebrity
faces to gaussian noise. The dataset is a low-resolution (32 × 32) version of the CelebA-HQ dataset that has been quantized to 2 bits per color channel.



In [ ]:
visualize_q3_data()

## Part (a): RealNVP [40pt]

Use the affine coupling flow from RealNVP and a form of [data-dependent initialization](https://arxiv.org/abs/1602.07868) that normalizes activations from an initial forward pass with a minibatch.
We describe a reference architecture below that can achieve good performance, but you are free
and encouraged to use different architectures and/or structure your code differently.



```
Preprocess() # dequantization, logit trick from RealNVP (Dinh et al) Section 4.1

for _ in range(4):
  AffineCouplingWithCheckerboard() # Figure 3 in Dinh et al - (left)
  ActNorm() # described in Glow (Kingma & Dhariwal) Section 3.1
Squeeze(), # [b, c, h, w] --> [b, c*4, h//2, w//2]

for _ in range(3):
  AffineCouplingWithChannel()
  ActNorm()
Unsqueeze(), # [b, c*4, h//2, w//2] --> [b, c, h, w]

for _ in range(3):
  AffineCouplingWithCheckerboard()
  ActNorm()
```



Here, we provide pseudocode for the basic structure of each coupling layer. 



```
# pseudocode
ResnetBlock: n_filters
  h = x
  h = conv2d(n_filters, n_filters, (1,1), stride=1, padding=0)(h)
  h = relu(h)
  h = conv2d(n_filters, n_filters, (3,3), stride=1, padding=1)(h)
  h = relu(h)
  h = conv2d(n_filters, n_filters, (1,1), stride=1, padding=0)(h)
  return h + x

SimpleResnet: n_filters = 256, n_blocks = 8, n_out
  conv2d(in_channels, n_filters=128, (3,3), stride=1, padding=1)
  apply 8 ResnetBlocks with n_filters=128
  relu()
  conv2d(n_filters, n_filters=n_out, (3,3), stride=1, padding=1)
  
AffineCoupling(x, mask):
  x_ = x * mask
  log_s, t = torch.chunk(SimpleResnet(x_), 2, dim=1)
  # todo: calculate log_scale, as done in Q1(b)
  t = t * (1.0 - mask)
  log_scale = log_scale * (1.0 - mask)
  z = x * torch.exp(log_scale) + t
  log_det_jacobian = log_scale
  return z, log_det_jacobian
```



We use an Adam Optimizer with a warmup over 200 steps till a learning rate of 5e-4. We didn’t decay the learning rate but it is a generally recommended practice while training generative models. We show samples from our model at the end of training, as well as interpolations between several images in our dataset.  Our reference solution achieves a test loss of about 0.40 bits/dim after 100 epochs. 



**You will provide these deliverables**


1.   Over the course of training, record the average negative log-likelihood (nats / dim) of the training data (per minibatch) and test data (for your entire test set). Code is provided that automatically plots the training curves. 
2.   Report the final test set performance of your final model
3. 100 samples from the final trained model
4. 5 rows of interpolations between images in the test set; each row consists of the start and end images, and 4 intermediate ones. You should return 30 total images. 



### Solution
Fill out the function below and return the necessary arguments. Feel free to create more cells if need be.

In [ ]:
import pickle
import time

import IPython
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from torch.autograd import Variable
from torchvision.utils import make_grid, save_image
from tqdm import tqdm, tqdm_notebook, trange

device = "cuda"

# conv2d with weight normalization...
class WeightNormConv2d(nn.Module):
    def __init__(self, in_dim, out_dim, kernel_size, stride=1, padding=0, bias=True):
        super(WeightNormConv2d, self).__init__()
        self.conv = nn.utils.weight_norm(
            nn.Conv2d(
                in_dim, out_dim, kernel_size, stride=stride, padding=padding, bias=bias
            )
        )

    def forward(self, x):
        return self.conv(x)


class ActNorm(nn.Module):
    def __init__(self, n_channels):
        super(ActNorm, self).__init__()
        self.log_scale = nn.Parameter(
            torch.zeros(1, n_channels, 1, 1), requires_grad=True
        )
        self.shift = nn.Parameter(torch.zeros(1, n_channels, 1, 1), requires_grad=True)
        self.n_channels = n_channels
        self.initialized = False

    def forward(self, x, reverse=False):
        if reverse:
            return (x - self.shift) * torch.exp(-self.log_scale), self.log_scale
        else:
            if not self.initialized:
                self.shift.data = -torch.mean(x, dim=[0, 2, 3], keepdim=True)
                self.log_scale.data = -torch.log(
                    torch.std(
                        x.permute(1, 0, 2, 3).reshape(self.n_channels, -1), dim=1
                    ).reshape(1, self.n_channels, 1, 1)
                )
                self.initialized = True
                result = x * torch.exp(self.log_scale) + self.shift
            return x * torch.exp(self.log_scale) + self.shift, self.log_scale


class ResnetBlock(nn.Module):
    def __init__(self, dim):
        super(ResnetBlock, self).__init__()
        self.block = nn.Sequential(
            WeightNormConv2d(dim, dim, (1, 1), stride=1, padding=0),
            nn.ReLU(),
            WeightNormConv2d(dim, dim, (3, 3), stride=1, padding=1),
            nn.ReLU(),
            WeightNormConv2d(dim, dim, (1, 1), stride=1, padding=0),
        )

    def forward(self, x):
        return x + self.block(x)


class SimpleResnet(nn.Module):
    def __init__(self, in_channels=3, out_channels=6, n_filters=128, n_blocks=8):
        super(SimpleResnet, self).__init__()
        layers = [
            WeightNormConv2d(in_channels, n_filters, (3, 3), stride=1, padding=1),
            nn.ReLU(),
        ]
        for _ in range(n_blocks):
            layers.append(ResnetBlock(n_filters))
        layers.append(nn.ReLU())
        layers.append(
            WeightNormConv2d(n_filters, out_channels, (3, 3), stride=1, padding=1)
        )
        self.resnet = nn.Sequential(*layers)

    def forward(self, x):
        return self.resnet(x)


class AffineCheckerboardTransform(nn.Module):
    def __init__(self, type=1.0):
        super(AffineCheckerboardTransform, self).__init__()
        self.mask = self.build_mask(type=type)
        self.scale = nn.Parameter(torch.zeros(1), requires_grad=True)
        self.scale_shift = nn.Parameter(torch.zeros(1), requires_grad=True)
        self.resnet = SimpleResnet()

    def build_mask(self, type=1.0):
        # if type == 1.0, the top left corner will be 1.0
        # if type == 0.0, the top left corner will be 0.0
        mask = np.arange(32).reshape(-1, 1) + np.arange(32)
        mask = np.mod(type + mask, 2)
        mask = mask.reshape(-1, 1, 32, 32)
        return torch.tensor(mask.astype("float32")).to(device)

    def forward(self, x, reverse=False):
        # returns transform(x), log_det
        batch_size, n_channels, _, _ = x.shape
        mask = self.mask.repeat(batch_size, 1, 1, 1)
        x_ = x * mask

        log_s, t = self.resnet(x_).split(n_channels, dim=1)
        log_s = self.scale * torch.tanh(log_s) + self.scale_shift
        t = t * (1.0 - mask)
        log_s = log_s * (1.0 - mask)

        if reverse:  # inverting the transformation
            x = (x - t) * torch.exp(-log_s)
        else:
            x = x * torch.exp(log_s) + t
        return x, log_s


class AffineChannelTransform(nn.Module):
    def __init__(self, modify_top):
        super(AffineChannelTransform, self).__init__()
        self.modify_top = modify_top
        self.scale = nn.Parameter(torch.zeros(1), requires_grad=True)
        self.scale_shift = nn.Parameter(torch.zeros(1), requires_grad=True)
        self.resnet = SimpleResnet(in_channels=6, out_channels=12)

    def forward(self, x, reverse=False):
        n_channels = x.shape[1]
        if self.modify_top:
            on, off = x.split(n_channels // 2, dim=1)
        else:
            off, on = x.split(n_channels // 2, dim=1)
        log_s, t = self.resnet(off).split(n_channels // 2, dim=1)
        log_s = self.scale * torch.tanh(log_s) + self.scale_shift

        if reverse:  # inverting the transformation
            on = (on - t) * torch.exp(-log_s)
        else:
            on = on * torch.exp(log_s) + t

        if self.modify_top:
            return torch.cat([on, off], dim=1), torch.cat(
                [log_s, torch.zeros_like(log_s)], dim=1
            )
        else:
            return torch.cat([off, on], dim=1), torch.cat(
                [torch.zeros_like(log_s), log_s], dim=1
            )


class RealNVP(nn.Module):
    def __init__(self):
        super(RealNVP, self).__init__()

        self.prior = torch.distributions.Normal(
            torch.tensor(0.0).to(device), torch.tensor(1.0).to(device)
        )
        # self.prior = torch.distributions.Normal(torch.tensor(0.).to(device), torch.tensor(1.).to(device))
        self.checker_transforms1 = nn.ModuleList(
            [
                AffineCheckerboardTransform(1.0),
                ActNorm(3),
                AffineCheckerboardTransform(0.0),
                ActNorm(3),
                AffineCheckerboardTransform(1.0),
                ActNorm(3),
                AffineCheckerboardTransform(0.0),
            ]
        )

        self.channel_transforms = nn.ModuleList(
            [
                AffineChannelTransform(True),
                ActNorm(12),
                AffineChannelTransform(False),
                ActNorm(12),
                AffineChannelTransform(True),
            ]
        )

        self.checker_transforms2 = nn.ModuleList(
            [
                AffineCheckerboardTransform(1.0),
                ActNorm(3),
                AffineCheckerboardTransform(0.0),
                ActNorm(3),
                AffineCheckerboardTransform(1.0),
            ]
        )

    def squeeze(self, x):
        # C x H x W -> 4C x H/2 x W/2
        [B, C, H, W] = list(x.size())
        x = x.reshape(B, C, H // 2, 2, W // 2, 2)
        x = x.permute(0, 1, 3, 5, 2, 4)
        x = x.reshape(B, C * 4, H // 2, W // 2)
        return x

    def undo_squeeze(self, x):
        #  4C x H/2 x W/2  ->  C x H x W
        [B, C, H, W] = list(x.size())
        x = x.reshape(B, C // 4, 2, 2, H, W)
        x = x.permute(0, 1, 4, 2, 5, 3)
        x = x.reshape(B, C // 4, H * 2, W * 2)
        return x

    def g(self, z):
        # z -> x (inverse of f)
        x = z
        for op in reversed(self.checker_transforms2):
            x, _ = op.forward(x, reverse=True)
        x = self.squeeze(x)
        for op in reversed(self.channel_transforms):
            x, _ = op.forward(x, reverse=True)
        x = self.undo_squeeze(x)
        for op in reversed(self.checker_transforms1):
            x, _ = op.forward(x, reverse=True)
        return x

    def f(self, x):
        # maps x -> z, and returns the log determinant (not reduced)
        z, log_det = x, torch.zeros_like(x)
        for op in self.checker_transforms1:
            z, delta_log_det = op.forward(z)
            log_det += delta_log_det
        z, log_det = self.squeeze(z), self.squeeze(log_det)
        for op in self.channel_transforms:
            z, delta_log_det = op.forward(z)
            log_det += delta_log_det
        z, log_det = self.undo_squeeze(z), self.undo_squeeze(log_det)
        for op in self.checker_transforms2:
            z, delta_log_det = op.forward(z)
            log_det += delta_log_det
        return z, log_det

    def log_prob(self, x):
        z, log_det = self.f(x)
        return torch.sum(log_det, [1, 2, 3]) + torch.sum(
            self.prior.log_prob(z), [1, 2, 3]
        )

    def sample(self, num_samples):
        z = self.prior.sample([num_samples, 3, 32, 32])
        return self.g(z)


class Solver(object):
    def __init__(self, learning_rate=5e-4, n_epochs=128):
        self.train_loader, self.val_loader = self.create_loaders()
        self.log_interval = 100
        self.n_epochs = n_epochs
        self.learning_rate = learning_rate
        self.n_batches_in_epoch = len(self.train_loader)

    def build(self):
        self.flow = RealNVP().to(device)
        self.optimizer = torch.optim.Adam(self.flow.parameters(), lr=self.learning_rate)

    def create_loaders(self):
        train, test = get_q3_data()
        train = np.transpose(train, axes=[0, 3, 1, 2])  # NCHW 20000 x 3 x 32 x 32
        test = np.transpose(test, axes=[0, 3, 1, 2])  # NCHW 6838 x 3 x 32 x 32
        train_loader = torch.utils.data.DataLoader(
            train, batch_size=64, shuffle=False, pin_memory=False
        )
        test_loader = torch.utils.data.DataLoader(
            test, batch_size=64, shuffle=False, pin_memory=False
        )
        return train_loader, test_loader

    def preprocess(self, x, reverse=False, dequantize=True):
        if reverse:  # doesn't map back to [0, 4]
            x = 1.0 / (1 + torch.exp(-x))
            x -= 0.05
            x /= 0.9
            return x
        else:
            # dequantization
            if dequantize:
                x += torch.distributions.Uniform(0.0, 1.0).sample(x.shape).to(device)
            x /= 4.0

            # logit operation
            x *= 0.9
            x += 0.05
            logit = torch.log(x) - torch.log(1.0 - x)
            log_det = (
                torch.nn.functional.softplus(logit)
                + torch.nn.functional.softplus(-logit)
                + torch.log(torch.tensor(0.9))
                - torch.log(torch.tensor(4.0))
            )
            return logit, torch.sum(log_det, dim=(1, 2, 3))

    def train(self):
        train_losses = []
        val_losses = []
        for epoch_i in range(self.n_epochs):
            epoch_i += 1

            self.flow.train()
            self.batch_loss_history = []

            for batch_i, image in enumerate(
                tqdm_notebook(self.train_loader, desc="Batch", leave=False)
            ):

                batch_i += 1
                # [batch_size, 3, 32, 32]
                image = Variable(image).to(device)
                logit_x, log_det = self.preprocess(image.float())
                log_prob = self.flow.log_prob(logit_x)
                log_prob += log_det

                batch_loss = -torch.mean(log_prob) / (3.0 * 32.0 * 32.0)

                self.optimizer.zero_grad()
                batch_loss.backward()
                self.optimizer.step()

                batch_loss = float(batch_loss.data)
                self.batch_loss_history.append(batch_loss)

            epoch_loss = np.mean(self.batch_loss_history)
            tqdm.write(f"Epoch {epoch_i} Loss: {epoch_loss:.2f}")

            if epoch_i % 10 == 0:
                self.save_model("realnvp{}.model".format(str(epoch_i)))
            train_losses.append(epoch_loss)
            val_losses.append(self.get_loss(self.val_loader))
            np.save("train_losses.npy", np.array(train_losses))
            np.save("val_losses.npy", np.array(val_losses))

        self.save_model("realnvp_final.model")
        return train_losses, val_losses

    def get_loss(self, loader):
        """Compute error on provided data set"""
        errors = []
        # cuda.synchronize()
        start = time.time()

        self.flow.eval()

        for image in loader:
            with torch.no_grad():
                image = image.to(device)
                logit_x, log_det = self.preprocess(image.float())
                log_prob = self.flow.log_prob(logit_x)
                log_prob += log_det

                loss = -torch.mean(log_prob) / (3.0 * 32.0 * 32.0)
                error = float(loss.data)
                errors.append(error)

        # cuda.synchronize()
        time_test = time.time() - start
        log_string = f"Calc done! | It took {time_test:.1f}s | "
        log_string += f"Loss: {np.mean(errors):.2f}"
        tqdm.write(log_string)
        return np.mean(errors)

    def sample(self, num_samples):
        with torch.no_grad():
            raw_samples = self.flow.sample(num_samples).cpu()
            samples = self.preprocess(raw_samples, reverse=True)
            return samples.cpu().numpy()

    def interpolate(self):
        self.flow.eval()
        good = [5, 13, 16, 19, 22]
        indices = []
        for index in good:
            indices.append(index * 2)
            indices.append(index * 2 + 1)
        with torch.no_grad():
            actual_images = next(iter(self.val_loader))[indices].to("cpu")
            assert actual_images.shape[0] % 2 == 0
            logit_actual_images, _ = self.preprocess(
                actual_images.float(), dequantize=False
            )
            latent_images, _ = self.flow.f(logit_actual_images)
            latents = []
            for i in range(0, actual_images.shape[0], 2):
                a = latent_images[i : i + 1]
                b = latent_images[i + 1 : i + 2]
                diff = (b - a) / 5.0
                latents.append(a)
                for j in range(1, 5):
                    latents.append(a + diff * float(j))
                latents.append(b)
            latents = torch.cat(latents, dim=0)
            logit_results = self.flow.g(latents)
            results = self.preprocess(logit_results, reverse=True)
            return results.cpu().numpy()

    def save_model(self, filename):
        torch.save(self.flow, filename)

    def load_model(self, filename):
        self.flow = torch.load(filename, map_location="cpu")

In [ ]:
def q3_a(train_data, test_data):
    """
    train_data: A (n_train, H, W, 3) uint8 numpy array of quantized images with values in {0, 1, 2, 3}
    test_data: A (n_test, H, W, 3) uint8 numpy array of binary images with values in {0, 1, 2, 3}

    Returns
    - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
    - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
    - a numpy array of size (100, H, W, 3) of samples with values in [0, 1]
    - a numpy array of size (30, H, W, 3) of interpolations with values in [0, 1].
    """

    """ YOUR CODE HERE """
    solver = Solver(n_epochs=1)
    solver.build()
    solver.load_model("realnvp_final.model")
    train_losses = np.load("train_losses.npy")
    test_losses = np.load("val_losses.npy")
    samples = np.transpose(solver.sample(100), axes=[0, 2, 3, 1])
    interpolations = np.transpose(solver.interpolate(), axes=[0, 2, 3, 1])
    return train_losses, test_losses, samples, interpolations

### Results

Once you've implemented `q3_a`, execute the cells below to visualize and save your results



In [ ]:
solver = Solver(n_epochs=100)
solver.build()
train, val = solver.train()

In [ ]:
q3_save_results(q3_a, "a")

## Part (b): Exploring bad masking schemes [10pt]

As shown in lecture, the choice of masking or partitioning scheme affects the quality of the final flow model. In this part, design a *bad* masking scheme and train a model with it on the same CelebA dataset. 

**Provide the same deliverables as part (a)**

### Solution

In [ ]:
class AffineHalfHalfTransform(nn.Module):
    def __init__(self, type):
        super(AffineHalfHalfTransform, self).__init__()
        self.mask = self.build_mask(type=type)
        self.scale = nn.Parameter(torch.zeros(1), requires_grad=True)
        self.scale_shift = nn.Parameter(torch.zeros(1), requires_grad=True)
        self.resnet = SimpleResnet()

    def build_mask(self, type):
        # if type == "top", the top half is 1s
        # if type == "bottom", the bottom half is 1s
        # if type == "left", left half is 1s
        # if type == right", right half is 1s
        assert type in {"top", "bottom", "left", "right"}
        if type == "bottom":
            mask = np.concatenate(
                [np.zeros([1, 1, 16, 32]), np.ones([1, 1, 16, 32])], axis=2
            )
        elif type == "top":
            mask = np.concatenate(
                [np.ones([1, 1, 16, 32]), np.zeros([1, 1, 16, 32])], axis=2
            )
        elif type == "left":
            mask = np.concatenate(
                [np.ones([1, 1, 32, 16]), np.zeros([1, 1, 32, 16])], axis=3
            )
        elif type == "right":
            mask = np.concatenate(
                [np.zeros([1, 1, 32, 16]), np.ones([1, 1, 32, 16])], axis=3
            )
        else:
            raise NotImplementedError
        return torch.tensor(mask.astype("float32")).to(device)

    def forward(self, x, reverse=False):
        # returns transform(x), log_det
        batch_size, n_channels, _, _ = x.shape
        mask = self.mask.repeat(batch_size, 1, 1, 1)
        x_ = x * mask

        log_s, t = self.resnet(x_).split(n_channels, dim=1)
        log_s = self.scale * torch.tanh(log_s) + self.scale_shift
        t = t * (1.0 - mask)
        log_s = log_s * (1.0 - mask)

        if reverse:  # inverting the transformation
            x = (x - t) * torch.exp(-log_s)
        else:
            x = x * torch.exp(log_s) + t
        return x, log_s


class BadRealNVP(nn.Module):
    def __init__(self):
        super(BadRealNVP, self).__init__()

        self.prior = torch.distributions.Normal(
            torch.tensor(0.0).to(device), torch.tensor(1.0).to(device)
        )
        self.transforms = nn.ModuleList(
            [
                AffineHalfHalfTransform("top"),
                ActNorm(3),
                AffineHalfHalfTransform("bottom"),
                ActNorm(3),
                AffineHalfHalfTransform("left"),
                ActNorm(3),
                AffineHalfHalfTransform("right"),
                ActNorm(3),
                AffineHalfHalfTransform("top"),
                ActNorm(3),
                AffineHalfHalfTransform("bottom"),
                ActNorm(3),
                AffineHalfHalfTransform("left"),
                ActNorm(3),
                AffineHalfHalfTransform("right"),
            ]
        )

    def g(self, z):
        # z -> x (inverse of f)
        x = z
        for op in reversed(self.transforms):
            x, _ = op.forward(x, reverse=True)
        return x

    def f(self, x):
        # maps x -> z, and returns the log determinant (not reduced)
        z, log_det = x, torch.zeros_like(x)
        for op in self.transforms:
            z, delta_log_det = op.forward(z)
            log_det += delta_log_det
        return z, log_det

    def log_prob(self, x):
        z, log_det = self.f(x)
        return torch.sum(log_det, [1, 2, 3]) + torch.sum(
            self.prior.log_prob(z), [1, 2, 3]
        )

    def sample(self, num_samples):
        z = self.prior.sample([num_samples, 3, 32, 32])
        return self.g(z)


class BadSolver(Solver):
    def __init__(self, learning_rate=5e-4, n_epochs=128):
        super(BadSolver, self).__init__(learning_rate, n_epochs)

    def build(self):
        self.flow = BadRealNVP().to(device)
        self.optimizer = torch.optim.Adam(self.flow.parameters(), lr=self.learning_rate)

    def train(self):
        train_losses = []
        val_losses = []
        for epoch_i in range(self.n_epochs):
            epoch_i += 1

            self.flow.train()
            self.batch_loss_history = []

            for batch_i, image in enumerate(
                tqdm_notebook(self.train_loader, desc="Batch", leave=False)
            ):

                batch_i += 1
                # [batch_size, 3, 32, 32]
                image = Variable(image).to(device)
                logit_x, log_det = self.preprocess(image.float())
                log_prob = self.flow.log_prob(logit_x)
                log_prob += log_det

                batch_loss = -torch.mean(log_prob) / (3.0 * 32.0 * 32.0)

                self.optimizer.zero_grad()
                batch_loss.backward()
                self.optimizer.step()

                batch_loss = float(batch_loss.data)
                self.batch_loss_history.append(batch_loss)

            epoch_loss = np.mean(self.batch_loss_history)
            tqdm.write(f"Epoch {epoch_i} Loss: {epoch_loss:.2f}")

            if epoch_i % 10 == 0:
                self.save_model("badrealnvp{}.model".format(str(epoch_i)))
            train_losses.append(epoch_loss)
            val_losses.append(self.get_loss(self.val_loader))
            np.save("bad_train_losses.npy", np.array(train_losses))
            np.save("bad_val_losses.npy", np.array(val_losses))

        self.save_model("badrealnvp_final.model")
        np.save("bad_train_losses.npy", np.array(train_losses))
        np.save("bad_val_losses.npy", np.array(val_losses))
        return train_losses, val_losses

In [ ]:
def q3_b(train_data, test_data):
    """
    train_data: A (n_train, H, W, 3) uint8 numpy array of quantized images with values in {0, 1, 2, 3}
    test_data: A (n_test, H, W, 3) uint8 numpy array of binary images with values in {0, 1, 2, 3}

    Returns
    - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
    - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
    - a numpy array of size (100, 3, H, W) of samples with values in [0, 1]
    - a numpy array of size (30, 3, H, W) of interpolations with values in [0, 1].
    """

    """ YOUR CODE HERE """
    solver = BadSolver(n_epochs=1)
    solver.build()
    solver.load_model("badrealnvp_final.model")
    train_losses = np.load("bad_train_losses.npy")
    test_losses = np.load("bad_val_losses.npy")
    samples = np.transpose(solver.sample(100), axes=[0, 2, 3, 1])
    interpolations = np.transpose(solver.interpolate(), axes=[0, 2, 3, 1])
    return train_losses, test_losses, samples, interpolations

### Results

Once you've implemented `q3_b`, execute the cells below to visualize and save your results



In [ ]:
solver = BadSolver(n_epochs=100)
solver.build()
train, val = solver.train()

In [ ]:
q3_save_results(q3_b, "b")

# Question 4: Bonus Questions (Optional)

## Q4(a): Multiscale RealNVP [10pt]
Following the alternating checkerboard, channel squeeze, and channel partition scheme used in https://arxiv.org/abs/1605.08803, learn a multiscale RealNVP model on our CelebA dataset used in Question 3. We expect the same deliverables as in Question 3; additionally, make sure to include analysis on any difference in qualitative or quantitative differences you see in the results.

In [ ]:
def q4_a(train_data, test_data):
    """
    train_data: A (n_train, H, W, 3) uint8 numpy array of quantized images with values in {0, 1, 2, 3}
    test_data: A (n_test, H, W, 3) uint8 numpy array of binary images with values in {0, 1, 2, 3}

    Returns
    - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
    - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
    - a numpy array of size (100, 3, H, W) of samples with values in [0, 1]
    - a numpy array of size (30, 3, H, W) of interpolations with values in [0, 1].
    """

    """ YOUR CODE HERE """

In [ ]:
q3_save_results(q4_a, "bonus_a")

## Q4(b): Glow [5pt]
The Glow paper introduces the invertible 1x1 convolution as a replacement for the permutation scheme that follows the channel squeezing step. Use this in your model from Question 3, and report the same deliverables. Make sure to include analysis on any difference in qualitative or quantitative differences you see in the results.

In [ ]:
def q4_b(train_data, test_data):
    """
    train_data: A (n_train, H, W, 3) uint8 numpy array of quantized images with values in {0, 1, 2, 3}
    test_data: A (n_test, H, W, 3) uint8 numpy array of binary images with values in {0, 1, 2, 3}

    Returns
    - a (# of training iterations,) numpy array of train_losses evaluated every minibatch
    - a (# of epochs + 1,) numpy array of test_losses evaluated once at initialization and after each epoch
    - a numpy array of size (100, 3, H, W) of samples with values in [0, 1]
    - a numpy array of size (30, 3, H, W) of interpolations with values in [0, 1].
    """

    """ YOUR CODE HERE """

In [ ]:
q3_save_results(q4_b, "bonus_b")

# File Saving Utility


In [ ]:
# one way to save files
import os
import os.path as osp

from google.colab import files

for fname in os.listdir("results"):
    files.download("results/" + fname)